# Parameter search

In [1]:
import cv2
import numpy as np

In [2]:
videofile = 'videos/test_bright.mp4'

In [3]:
cap = cv2.VideoCapture(videofile)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
print("length:\t", length)
print("width:\t",width)
print("height:\t",height)
print("fps:\t",fps)
print("frame_no:\t",frame_no)
cap.release()

length:	 1359
width:	 1920
height:	 1080
fps:	 23.976029850951072
frame_no:	 0


In [4]:
cap = cv2.VideoCapture(videofile)
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
    
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:

        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) == 130:
            frame_on = frame.copy()
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) == 133:
            frame_off = frame.copy()
    else: 
        break
cap.release()

def nothing(x):
    pass
# Create a black image, a window
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 800,1200)
 
# create trackbars for color change
cv2.createTrackbar('blur_size','image',0,50,nothing)
cv2.createTrackbar('threshold','image',0,255,nothing)
cv2.createTrackbar('m_close_size','image',1,50,nothing)
cv2.createTrackbar('m_open_size','image',1,50,nothing)
cv2.createTrackbar('d_area_min','image',0,1000,nothing)
cv2.createTrackbar('d_area_max','image',0,100000,nothing)
cv2.createTrackbar('d_circularity','image',0,100,nothing)
cv2.createTrackbar('d_convexity','image',0,100,nothing)
cv2.createTrackbar('d_inertia','image',0,100,nothing)


# create switch for ON/OFF functionality
switch = '0 : OFF \t1 : ON'
cv2.createTrackbar(switch, 'image',0,1,nothing)
cv2.createTrackbar('blur', 'image',0,1,nothing)
cv2.createTrackbar('hsv_or_gray', 'image',0,1,nothing)
cv2.createTrackbar('diff', 'image',0,1,nothing)
cv2.createTrackbar('thres', 'image',0,1,nothing)
cv2.createTrackbar('morph_close', 'image',0,1,nothing)
cv2.createTrackbar('morph_open', 'image',0,1,nothing)
cv2.createTrackbar('detect', 'image',0,1,nothing)

while(1):

    if cv2.getTrackbarPos(switch,'image'):
        frame = frame_on
    else:
        frame = frame_off

    f_on = frame_on.copy()
    f_off = frame_off.copy()

    if cv2.getTrackbarPos('blur','image'):
        blur_size = cv2.getTrackbarPos('blur_size','image') * 2 + 1
        frame = cv2.GaussianBlur(frame, (blur_size,blur_size), 0)
        f_on = cv2.GaussianBlur(f_on, (blur_size,blur_size), 0)
        f_off = cv2.GaussianBlur(f_off, (blur_size,blur_size), 0)
    
    if cv2.getTrackbarPos('hsv_or_gray','image'):
        f_on = cv2.cvtColor(frame_on,cv2.COLOR_BGR2HSV)
        f_on[:,:,0] = 0
        f_on[:,:,1] = 0
        f_off = cv2.cvtColor(frame_off,cv2.COLOR_BGR2HSV)
        f_off[:,:,0] = 0
        f_off[:,:,1] = 0

    f_on = cv2.cvtColor(f_on, cv2.COLOR_BGR2GRAY)
    f_off = cv2.cvtColor(f_off, cv2.COLOR_BGR2GRAY)
    
    if cv2.getTrackbarPos('diff','image'):
        frame = cv2.absdiff(f_on, f_off)
        
    if cv2.getTrackbarPos('thres','image'):
        ret, frame = cv2.threshold(frame,cv2.getTrackbarPos('threshold','image'),255, cv2.THRESH_BINARY)
        

    if cv2.getTrackbarPos('morph_close','image'):
        kernel_size = cv2.getTrackbarPos('m_close_size','image') * 2 + 1
        kernel = np.ones((kernel_size,kernel_size),np.uint8)
        frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel)

    if cv2.getTrackbarPos('morph_open','image'):
        kernel_size = cv2.getTrackbarPos('m_open_size','image') * 2 + 1
        kernel = np.ones((kernel_size,kernel_size),np.uint8)
        frame = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)


    # Set up the SimpleBlobdetector with default parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 0
    params.maxThreshold = 256
        
    # Filter by Area.
    params.filterByArea = True
    params.minArea = cv2.getTrackbarPos('d_area_min','image')
    params.maxArea = cv2.getTrackbarPos('d_area_max','image')
        
    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = cv2.getTrackbarPos('d_circularity','image') / 100
        
    # Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity = cv2.getTrackbarPos('d_convexity','image') / 100
        
    # Filter by Inertia
    params.filterByInertia =True
    params.minInertiaRatio = cv2.getTrackbarPos('d_inertia','image') / 100
        
    detector = cv2.SimpleBlobDetector_create(params)
    
    if cv2.getTrackbarPos('detect','image'):
        keypoints = detector.detect(~frame)
        frame = cv2.drawKeypoints(frame, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    
    cv2.imshow('image2',frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()